In [1]:
import os
import requests
import time
import boto3
import pandas as pd
import jupyter_black
from io import BytesIO
from io import StringIO
from tqdm.notebook import tqdm
import altair as alt

In [2]:
jupyter_black.load()
pd.options.display.max_columns = 2000
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [3]:
current_year = pd.Timestamp("now").year

In [21]:
# Headers for requests
headers = {
    "sec-ch-ua": '"Chromium";v="124", "Google Chrome";v="124", "Not-A.Brand";v="99"',
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36",
    "sec-ch-ua-platform": '"macOS"',
}

In [22]:
stars = [
    "Shohei Ohtani",
]

In [35]:
batter_list = requests.get(
    f"https://bdfed.stitch.mlbinfra.com/bdfed/stats/player?&env=prod&season={current_year}&sportId=1&stats=season&group=hitting&gameType=R&offset=0&sortStat=homeRuns&order=desc&teamId=119",
    headers=headers,
)

pitcher_list = requests.get(
    f"https://bdfed.stitch.mlbinfra.com/bdfed/stats/player?&env=prod&season={current_year}&sportId=1&stats=season&group=pitching&gameType=R&offset=0&sortStat=homeRuns&order=desc&teamId=119",
    headers=headers,
)

In [ ]:
batters = pd.DataFrame(batter_list.json()["stats"])
pitchers = pd.DataFrame(pitcher_list.json()["stats"])

In [44]:
# shohei_pitching = pitchers.query('playerName == "Shohei Ohtani"').iloc[0]

In [45]:
shohei = batters.query('playerName == "Shohei Ohtani"').iloc[0]

In [33]:
params = {
    "playerId": shohei["playerId"],
    "type": "",
    "season": current_year,
}

pitch_data = requests.get(
    "https://baseballsavant.mlb.com/player-viz/lookup",
    params=params,
    headers=headers,
).json()["data"]

pitch_df = pd.DataFrame(pitch_data).query("is_lastpitch == 1").fillna("")

In [34]:
pitch_df["game_date"] = pd.to_datetime(pitch_df["game_date"])
pitch_df = pitch_df.sort_values("game_date").copy()

In [35]:
pitch_df["pa_number"] = pitch_df.groupby(["batter_name"])["play_id"].cumcount()

In [ ]:
pitch_df["event_type"].value_counts()

In [60]:
shohei_homer_at_bats = pitch_df.query(
    "is_hit_into_play_basehit == 1 and events == 'Home run'"
)

In [67]:
shohei_homer_at_bats.to_clipboard()

In [62]:
# pitch_df.query("is_hit_into_play_basehit == 1")

In [63]:
# pitch_df.query("is_hit_into_play_basehit == 0").events.value_counts()

In [64]:
# alt.Chart(pitch_df).mark_tick(size=30, thickness=1).encode(
#     x=alt.X("pa_number:Q", axis=alt.Axis(grid=False)),
#     y=alt.Y("batter_name:O", axis=alt.Axis(grid=False)),
#     color=alt.condition(
#         "datum.events == 'home_run'",
#         alt.value("#005a9c"),
#         alt.value("#ccc"),
#     ),
#     tooltip=["pa_number:Q", "game_date:T", "events", "inning"],
# ).properties(width=600, height=100).configure_view(strokeWidth=0)

In [65]:
# pitch_df["batter_name_clean"] = (
#     pitch_df["batter_name"].str.split(", ", expand=True)[1]
#     + " "
#     + pitch_df["batter_name"].str.split(", ", expand=True)[0]
# )

In [43]:
positive = [
    "Hit by pitch",
    "Field error",
    # "Strikeout",
    # "Field out",
    "Single",
    "Walk",
    "Fielders choice",
    "Sac fly",
    # "Double play",
    # "Force out",
    "Double",
    "Home run",
    # "Grounded into double play",
    "Triple",
    # "Caught stealing second",
    # "Catcher interf",
    "Sac bunt",
    # "Pickoff caught stealing third",
]

pitch_df["positive_outcome"] = pitch_df["events"].isin(positive)

In [44]:
slim_cols = [
    "player_id",
    "batter_name_clean",
    "pa_number",
    "events",
    "event_type",
    "inning",
    "des",
    "game_date",
    "game_year",
    "is_hit_into_play",
    "is_hit_into_play_basehit",
    "positive_outcome",
]

In [45]:
pitch_df["game_date"] = pitch_df["game_date"].astype(str)
pitch_df["player_id"] = pitch_df["player_id"].astype(str)
pitch_df["game_year"] = pitch_df["game_year"].astype(str)
pitch_df["des"] = pitch_df["des"].str.replace(".   ", ". ").str.replace(".  ", ". ")

In [46]:
pitch_df["events"] = (
    pitch_df["events"]
    .str.strip()
    .str.replace("_", " ")
    .str.replace("2b", "second")
    .str.replace("3b", "third")
    .str.capitalize()
)

In [47]:
pitch_df["event_type"] = (
    pitch_df["event_type"].str.strip().str.replace("_", " ").str.capitalize()
)

In [48]:
pitch_df[slim_cols].to_json(
    "../data/batting/dodgers_player_atbat_lastpitch_outcome_current.json",
    indent=4,
    orient="records",
    lines=False,
)

In [49]:
pitch_df

,player_id,batter_name,pitcher_name,batter,pitcher,venue,team_batting,team_pitching,pitch_type,game_date,release_speed,event_type,is_lastpitch,release_pos_x,release_pos_y,release_pos_z,ba_numer,ba_denom,est_ba_numer,est_ba_denom,bacon_numer,bacon_denom,est_bacon_numer,est_bacon_denom,babip_numer,babip_denom,obp_numer,obp_denom,slg_numer,slg_denom,est_obp_numer,est_obp_denom,est_slg_numer,est_slg_denom,iso_numer,iso_denom,est_iso_numer,est_iso_denom,fangraphs_woba_numer,fangraphs_woba_denom,fangraphs_est_woba_numer,fangraphs_est_woba_denom,fangraphs_wobacon_numer,fangraphs_wobacon_denom,fangraphs_est_wobacon_numer,fangraphs_est_wobacon_denom,zone,api_gameday3_pitchzone_cd,des,events,game_type,stand,p_throws,home_team,away_team,home_team_id,away_team_id,batting_team_id,fielding_team_id,hit_location,bb_type,balls,strikes,pre_ball_count,pre_strike_count,game_year,year,api_break_x,api_break_x_arm,api_break_x_glove,api_break_x_batter_in,api_break_x_batter_out,api_break_z_induced,api_break_z_with_gravity,plate_x,plate_z,px,pz,py,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,hc_x_ft,hc_y_ft,pos1_person_id,fielder_2,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,sv_id,vx0,vy0,vz0,ax,ay,az,x0,y0,z0,sz_top,sz_bot,is_hit_into_play_basehit,is_hit_into_play,is_ab,is_pa,is_whiff,is_swing,is_take,is_strikeout,is_walk,is_hit_into_play_basehit_single,is_hit_into_play_basehit_double,is_hit_into_play_basehit_triple,is_hit_into_play_basehit_home_run,is_sac,is_sf,is_foul,play_id,hit_distance_sc,launch_angle,launch_speed,launch_direction,is_competitive,sweetspot_speed_mph,swing_length,hyper_speed,effective_speed,release_spin_rate,release_extension,game_pk,distance_projected,slg_value,iso_value,woba_value,woba_denom,estimated_slg_using_speedangle,h_launch_speedangle_code,babip_value,pos1_int_start_distance,pos2_int_start_distance,pos3_int_start_distance,pos4_int_start_distance,pos5_int_start_distance,pos6_int_start_distance,pos7_int_start_distance,pos8_int_start_distance,pos9_int_start_distance,launch_speed_angle,home_name_abbrev,venue_name,venue_id,touch1_fielder_pos,pitch_name,type,at_bat_number,pitch_number,estimated_ba_using_speedangle,home_score,away_score,home_score_diff,bat_score,fld_score,bat_score_diff,post_away_score,post_home_score,post_bat_score,post_fld_score,n_priorpa_thisgame_player_at_bat,n_thruorder_pitcher,if_fielding_alignment,of_fielding_alignment,spin_axis,home_win_exp,bat_win_exp,delta_home_win_exp,delta_run_exp,delta_pitcher_run_exp,unadj_delta_run_exp,unadj_delta_pitcher_run_exp,delev_delta_run_exp,delev_delta_pitcher_run_exp,is_pitch,b_name_display_first_last,age_pit_legacy,age_bat_legacy,age_pit,age_bat,is_hit_into_play_hardhit,is_speedangle_barrel,pitcher_days_since_prev_game,pitcher_days_until_next_game,batter_days_since_prev_game,batter_days_until_next_game,api_game_date_month_text,api_game_date_month_mm,playoff_series_name,series_game_number,arm_angle,description,stringer_hit_trajectory,league,pa_number,batter_name_clean,positive_outcome
565,660271,"Ohtani, Shohei","Imanaga, Shota",660271,684007,Tokyo Dome,LAD,CHC,FF,2025-03-18,93.1,Hit into play,1,2.45,54.29,5.46,0.0,1.0,0.047,1.0,0.0,1.0,0.047,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.047,1.0,0.055,1.0,0.0,1.0,0.008,1.0,0.000000,1.0,0.042000,1.0,0.000000,1.0,0.042,1.0,5,5,"Shohei Ohtani grounds out, second baseman Jon Berti to first baseman Michael Busch.",Field out,R,L,L,CHC,LAD,112,119,119,112,4.0,ground_ball,1,1,1,1,2025,2025,1.06,1.06,-1.06,1.06,-1.06,1.66,1.02,-0.14,2.61,-0.166684,0.103407,,,,,0,1,Top,148.47,156.14,67.2,123.4,684007,665804,683737,542932,807713,621020,664023,691718,663656,,-8.947780,-135.121062,-5.039529,15.106256,34.552133,-11.020534,2.164467,50.004234,5.305126,3.4,1.62,0.0,1.0,1.0,1.0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,f7d37a4f-9cf6-4172-ac23-bb66e23a5923,7.0,-20.0,76,34.0,1,81.6,8.7,88.0,92.1,2664,6.2,778563,7.0,0.0,0.0,0,1.0,0.04,2.0,0.0,,,,,,,,,,2.0,CHC,Tokyo Dome,2397,4.0,4-Seam Fastball,X,1,3,0.04,0,0,0,0,0,0,